<a href="https://colab.research.google.com/github/Eduardoarteagal/PMD-Taller1_2P/blob/main/Taller_PySpark_1_PMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=59e97cbac0acb235407d154ef56c8ac57daba2371da04af9a66ef6a358d4ddab
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

SpSession = SparkSession \
          .builder \
          .appName("Demo Spark") \
          .getOrCreate()

In [ ]:
SpContext = SpSession.sparkContext

In [ ]:
data = SpSession.read.csv('ventas.csv', header=True, inferSchema=True)
data.show(5)

+----------+----------+--------+-----------+
|id_cliente|  producto|cantidad|monto_total|
+----------+----------+--------+-----------+
|         1|producto_1|       2|       20.0|
|         2|producto_2|       1|       15.0|
|         3|producto_3|       3|       30.0|
|         4|producto_4|       2|       25.0|
|         5|producto_5|       1|       10.0|
+----------+----------+--------+-----------+
only showing top 5 rows



In [ ]:
data.printSchema()

root
 |-- id_cliente: integer (nullable = true)
 |-- producto: string (nullable = true)
 |-- cantidad: integer (nullable = true)
 |-- monto_total: double (nullable = true)



In [ ]:
data.columns

['id_cliente', 'producto', 'cantidad', 'monto_total']

In [ ]:
data.dtypes

[('id_cliente', 'int'),
 ('producto', 'string'),
 ('cantidad', 'int'),
 ('monto_total', 'double')]

In [ ]:
data.select(data.cantidad, data.monto_total).show(truncate=False)

+--------+-----------+
|cantidad|monto_total|
+--------+-----------+
|2       |20.0       |
|1       |15.0       |
|3       |30.0       |
|2       |25.0       |
|1       |10.0       |
|2       |30.0       |
|4       |40.0       |
|3       |35.0       |
|2       |20.0       |
|1       |15.0       |
|2       |20.0       |
|1       |15.0       |
|3       |30.0       |
|2       |25.0       |
|1       |10.0       |
|2       |30.0       |
|4       |40.0       |
|3       |35.0       |
|2       |20.0       |
|1       |15.0       |
+--------+-----------+
only showing top 20 rows



In [ ]:
#suma total de las columnas cantidad y monto_total
total_cantidad = data.groupBy().sum("cantidad").collect()[0][0]
total_valor = data.groupBy().sum("monto_total").collect()[0][0]

print("Ventas totales en cantidad:",total_cantidad)
print("Ventas totales en valor de venta:", total_valor)

Ventas totales en cantidad: 840
Ventas totales en valor de venta: 9600.0


In [ ]:
#filtrado cuantas veces se ha vendido el producto 1, 2, 3
cantidad_producto_1 = data.filter(data.producto == "producto_1").groupBy().sum("cantidad").collect()[0][0]
cantidad_producto_2 = data.filter(data.producto == "producto_2").groupBy().sum("cantidad").collect()[0][0]
cantidad_producto_3 = data.filter(data.producto == "producto_3").groupBy().sum("cantidad").collect()[0][0]

print("Cantidad de producto 1 vendido:", cantidad_producto_1)
print("Cantidad de producto 2 vendido:", cantidad_producto_2)
print("Cantidad de producto 3 vendido:", cantidad_producto_3)

Cantidad de producto 1 vendido: 6
Cantidad de producto 2 vendido: 3
Cantidad de producto 3 vendido: 9


In [ ]:
#ganacia total
from pyspark.sql.functions import sum as spark_sum, col
ventas = data.withColumn("ganancia", col("cantidad") * col("monto_total"))
ganancia = ventas.agg(spark_sum("ganancia")).collect()[0][0]
print(ventas.show())
print("Ganancia: ",ganancia)

+----------+-----------+--------+-----------+--------+
|id_cliente|   producto|cantidad|monto_total|ganancia|
+----------+-----------+--------+-----------+--------+
|         1| producto_1|       2|       20.0|    40.0|
|         2| producto_2|       1|       15.0|    15.0|
|         3| producto_3|       3|       30.0|    90.0|
|         4| producto_4|       2|       25.0|    50.0|
|         5| producto_5|       1|       10.0|    10.0|
|         6| producto_6|       2|       30.0|    60.0|
|         7| producto_7|       4|       40.0|   160.0|
|         8| producto_8|       3|       35.0|   105.0|
|         9| producto_9|       2|       20.0|    40.0|
|        10|producto_10|       1|       15.0|    15.0|
|        11|producto_11|       2|       20.0|    40.0|
|        12|producto_12|       1|       15.0|    15.0|
|        13|producto_13|       3|       30.0|    90.0|
|        14|producto_14|       2|       25.0|    50.0|
|        15|producto_15|       1|       10.0|    10.0|
|        1

In [ ]:

#ganancia del producto 1,2 y 3
ganancia_producto_1 = ventas.filter(ventas.producto == "producto_1").agg(spark_sum("ganancia")).collect()[0][0]
ganancia_producto_2 = ventas.filter(ventas.producto == "producto_2").agg(spark_sum("ganancia")).collect()[0][0]
ganancia_producto_3 = ventas.filter(ventas.producto == "producto_3").agg(spark_sum("ganancia")).collect()[0][0]
print("Ganancia de producto 1:" ,ganancia_producto_1)
print("Ganancia de producto 2:" ,ganancia_producto_2)
print("Ganancia de producto 3:" ,ganancia_producto_3)

Ganancia de producto 1: 120.0
Ganancia de producto 2: 45.0
Ganancia de producto 3: 270.0
